In [1]:
from solid import translate, rotate, union
from solid import cylinder, sphere
from solid import scad_render
from viewscad import Renderer

from datetime import datetime

import numpy as np

renderer = Renderer(openscad_exec='C:\Program Files\OpenSCAD\openscad.exe')

In [15]:
start_time = datetime.now()
%run Tree.py

t = Tree([0, 0, 0], [0, 0, 0])
t.render(4)
tn = t.getParts()
print(len(t.branches), len(tn))
renderer.render(union()(tn))
print(f'{(datetime.now() - start_time).total_seconds():0.3f}')

8 15


28.488


In [16]:
import pprint

config = t.getConfig()

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(config)

[   [   [0, 0, 0],
        [0, 13.229581387404254, -67.79962691117754],
        [50.0, 2, 1.5],
        [2]],
    [   [4.323578146832295, -10.594407635400241, 48.673043862729685],
        [0, 10.612544927364533, -45.83000840342717],
        [37.5, 1.5, 1.0],
        [1.5]],
    [   [9.13577804615213, -15.548089703716245, 85.53160821479081],
        [0, 19.922565298096693, 92.09119325798824],
        [25.0, 1.0, 0.5],
        [1.0]],
    [   [8.824928106131013, -7.035016923742063, 109.03545820064741],
        [0, 28.447267697293434, 260.596371082039],
        [12.5, 0.5, 0.0],
        [0.5]],
    [   [4.323578146832295, -10.594407635400241, 48.673043862729685],
        [0, 89.92495864395892, 194.77737181468683],
        [37.5, 1.125, 0.75],
        [1.125]],
    [   [-31.93604817945286, -20.159295844369836, 48.7221583013661],
        [0, 79.45584815802937, 317.3492504126506],
        [25.0, 0.75, 0.375],
        [0.75]],
    [   [-13.859103496237978, -36.811473177596795, 53.296987367326

In [ ]:
def decay(y_0, y_N, k):
    def y(x):
        return y_N - (y_N - y_0) * np.exp(- k * x)
    return y

In [ ]:
N = 10

L_0 = 50 * 0.3
L_N = 50
k_L =  - np.log(0.05) / N
L = decay(L_0, L_N, k_L)

r_0 = 0.5
r_N = 0.1
k_r =  - np.log(0.05) / N
r = decay(r_0, r_N, k_r)

theta_0 = 0
theta_N = 45
k_theta = - np.log(0.05) / N
theta = decay(0, 45, k_theta)

r1 = r_0
C_list = []
L_list = []

start_point = [0, 0, 0]
orientation = [0, 0, 0]

Theta = [orientation[1]]
Phi = [orientation[2]]
X = [start_point[0]]
Y = [start_point[1]]
Z = [start_point[2]]


for i in range(N):
    l = L(i) - sum(L_list)
    r2 = r(i)
    t = 5 - 5 * np.random.rand()
    phi = 5 - 5 * np.random.rand()
    z = l * np.cos(np.deg2rad(np.sum(Theta) + t))
    w = l * np.sin(np.deg2rad(np.sum(Theta) + t))
    x = w * np.cos(np.deg2rad(np.sum(Phi) + phi))
    y = w * np.sin(np.deg2rad(np.sum(Phi) + phi))
    
    print(f'L: {l:.03f} \t r: {r1:.03f} , {r2:.03f} \t t: {t:.03f}')
    
    
    c = translate([sum(X), sum(Y), sum(Z)])(
        rotate([0, sum(Theta) + t, sum(Phi) + phi])(
            cylinder(h=l, r1=r1, r2=r2)
        )
    )
        
    
    C_list.append(c.copy())
    L_list.append(l)
    r1 = r2
    
    Theta.append(t)
    Phi.append(phi)
    X.append(x)
    Y.append(y)
    Z.append(z)

In [ ]:
renderer.render(union()(*C_list))